In [1]:
import codecs
import numpy as np
import pandas as pd
from rc4 import rc4
from steganography import Steganography
from cnn import Net
import cv2

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
torch.cuda.is_available()

False

In [ ]:
class Net(nn.Module):

    def __init__(self):

        super(Net, self).__init__()

        # Covolutional Layers
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 5)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3)
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3)
        self.conv4 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 2)

        # Maxpooling Layer
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)

        # Fully Connected Layers
        self.fc1 = nn.Linear(in_features = 230400, out_features = 1000) # The number of input gained by "print("Flatten size: ", x.shape)" in below
        self.fc2 = nn.Linear(in_features = 1000,    out_features = 1000)
        self.fc3 = nn.Linear(in_features = 1000,    out_features = 2) # the output 136 in order to having 2 for each of the 68 keypoint (x, y) pairs

        # Dropouts
        self.drop1 = nn.Dropout(p = 0.1)
        self.drop2 = nn.Dropout(p = 0.2)
        self.drop3 = nn.Dropout(p = 0.3)
        self.drop4 = nn.Dropout(p = 0.4)
        self.drop5 = nn.Dropout(p = 0.5)
        self.drop6 = nn.Dropout(p = 0.6)




    def forward(self, x):

        # First - Convolution + Activation + Pooling + Dropout
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.drop1(x)
        #print("First size: ", x.shape)

        # Second - Convolution + Activation + Pooling + Dropout
        x = self.drop2(self.pool(F.relu(self.conv2(x))))
        #print("Second size: ", x.shape)

        # Third - Convolution + Activation + Pooling + Dropout
        x = self.drop3(self.pool(F.relu(self.conv3(x))))
        #print("Third size: ", x.shape)

        # Forth - Convolution + Activation + Pooling + Dropout
        x = self.drop4(self.pool(F.relu(self.conv4(x))))
        #print("Forth size: ", x.shape)

        # Flattening the layer
        x = x.flatten()
        #print("Flatten size: ", x.shape)

        # First - Dense + Activation + Dropout
        x = self.drop5(F.relu(self.fc1(x)))
        #print("First dense size: ", x.shape)

        # Second - Dense + Activation + Dropout
        x = self.drop6(F.relu(self.fc2(x)))
        #print("Second dense size: ", x.shape)

        # Final Dense Layer
        x = self.fc3(x)
        #print("Final dense size: ", x.shape)

        return x


In [ ]:
class rc4:

  def __init__(self, plaintext, key):
    self.plaintext = plaintext
    self.key = key
    self.ciphertext = " "
    self.MOD = 256
    self.S = list(range(self.MOD))

  def KSA(self):
    ''' Key Scheduling Algorithm (from wikipedia):
        for i from 0 to 255
            S[i] := i
        endfor
        j := 0
        for i from 0 to 255
            j := (j + S[i] + key[i mod keylength]) mod 256
            swap values of S[i] and S[j]
        endfor
    '''
    key_length = len(self.key)
    # create the array "S"
    # [0,1,2, ... , 255]
    j = 0
    for i in range(self.MOD):
        j = (j + self.S[i] + self.key[i % key_length]) % self.MOD
        self.S[i], self.S[j] = self.S[j], self.S[i]  # swap values

    return self.S


  def PRGA(self):
    ''' Psudo Random Generation Algorithm (from wikipedia):
        i := 0
        j := 0
        while GeneratingOutput:
            i := (i + 1) mod 256
            j := (j + S[i]) mod 256
            swap values of S[i] and S[j]
            K := S[(S[i] + S[j]) mod 256]
            output K
        endwhile
    '''
    i = 0
    j = 0
    while True:
        i = (i + 1) % self.MOD
        j = (j + self.S[i]) % self.MOD

        self.S[i], self.S[j] = self.S[j], self.S[i]  # swap values
        K = self.S[(self.S[i] + self.S[j]) % self.MOD]
        yield K


  def get_keystream(self):
    ''' Takes the encryption key to get the keystream using PRGA
        return object is a generator
    '''
    self.S = self.KSA()
    return self.PRGA()
    

  def encrypt_logic(self):
    ''' :key -> encryption key used for encrypting, as hex string
        :text -> array of unicode values/ byte string to encrpyt/decrypt
    '''
    # For plaintext key, use this
    self.key = [ord(c) for c in self.key]
    # If key is in hex:
    # key = codecs.decode(key, 'hex_codec')
    # key = [c for c in key]
    keystream = self.get_keystream()

    res = []
    for c in self.plaintext:
        val = ("%02X" % (c ^ next(keystream)))  # XOR and taking hex
        res.append(val)
    return ''.join(res)
    

  def encrypt(self):
    ''' :key -> encryption key used for encrypting, as hex string
        :plaintext -> plaintext string to encrpyt
    '''
    self.plaintext = [ord(c) for c in self.plaintext]
    return self.encrypt_logic()

    
  def decrypt(self):
    ''' :key -> encryption key used for encrypting, as hex string
        :ciphertext -> hex encoded ciphered text using RC4
    '''
    self.ciphertext = codecs.decode(self.ciphertext, 'hex_codec')
    res = encrypt_logic(self.key, self.ciphertext)
    return codecs.decode(res, 'hex_codec').decode('utf-8')

cipher = rc4("Rachit", 'not-so-random-key')

cipher.encrypt()

In [ ]:
class Steganography():

  def __init__(self, image, cipher, filename):
    self.img = image
    self.data = cipher
    self.pixel = 5.5
    self.list1 = [ ]
    self.filename = filename
    import numpy as np
    import cv2 as cv
    self.np = np
    self.cv = cv

  def BinaryTointeger(binary):  
      binary1 = binary  
      decimal, i, n = 0, 0, 0
      while(binary != 0):  
          dec = binary % 10
          decimal = decimal + dec * pow(2, i)  
          binary = binary//10
          i += 1
      return (decimal)     


  def message2binary(self):

    if type(self.data) == str:
      result= ''.join([ format(ord(i), "08b") for i in self.data ])

    elif type(self.data) == bytes or type(self.data) == self.np.ndarray:
      result= [ format(i, "08b") for i in self.data ]
      
    elif type(self.data) == int or type(self.data) == self.np.uint8:
      result=format(self.data, "08b")

    else:
      raise TypeError("Input type is not supported")
      
    return result

  def message2binaryA(self):
    if type(self.pixel) == str:
      result= ''.join([ format(ord(i), "08b") for i in self.pixel ])
      
    elif type(self.pixel) == bytes or type(self.pixel) == self.np.ndarray:
      result= [ format(i, "08b") for i in self.pixel ]
      
    elif type(self.pixel) == int or type(self.pixel) == self.np.uint8:
      result=format(self.pixel, "08b")

    else:
      raise TypeError("Input type is not supported")
      
    return result

  def encode_data(self):   
    if (len(self.data) == 0): 
      raise ValueError('Data is empty')
  
    
    no_bytes=(self.img.shape[0] * self.img.shape[1] * 3) // 8
    
    #print("Maximum bytes to encode:", no_bytes)
    
    if(len(self.data)>no_bytes):
        raise ValueError("Error encountered Insufficient bytes, Need Bigger Image or give Less Data !!")
    
    # Using the below as delimeter
    self.data +='*****'    
    
    data_binary=self.message2binary()
    #print(data_binary)
    data_len=len(data_binary)
    
    #print("The Length of Binary data",data_len)
    
    data_index = 0
    
    for i in self.img:
        for self.pixel in i:
            
          r, g, b = self.message2binaryA()
         # print(r)
         # print(g)
         # print(b)
          # print(pixel)
          if data_index < data_len:
              # hiding the data into LSB(Least Significant Bit) of Red Pixel
              #print("Original Binary",r)
              # print("The old pixel",pixel[0])
              self.pixel[0] = int(r[:-1] + data_binary[data_index], 2) #changing to binary after overwrriting the LSB bit of Red Pixel
              #print("Changed binary",r[:-1] + data_binary[data_index])
              
              data_index += 1
              self.list1.append(self.pixel[0])

          if data_index < data_len:
             # hiding the data into LSB of Green Pixel
              self.pixel[1] = int(g[:-1] + data_binary[data_index], 2) #changing to binary after overwrriting the LSB bit of Green Pixel
              data_index += 1
              self.list1.append(self.pixel[1])

          if data_index < data_len:
              # hiding the data into LSB of  Blue Pixel
              self.pixel[2] = int(b[:-1] + data_binary[data_index], 2) #changing to binary after overwrriting the LSB bit of Blue pixel
              data_index += 1
              self.list1.append(self.pixel[2])

              # if data is encoded, just breaking out of the Loop
          if data_index >= data_len:
              break

    self.cv.imwrite(self.filename, self.img)
    print("Encoded the data successfully and the image is successfully saved as ", self.filename)

    return self.img

#import cv2
#image=cv2.imread("00002.jpg")
#image.shape

#modifiedImg = Steganography(image, "3871E275B6CD")

#modifiedImage = modifiedImg.encode_data()

#import numpy as np

#modifiedImage.shape

In [3]:
cipher = rc4("Rachit", 'not-so-random-key')
ciphertext = cipher.encrypt()

In [4]:
ciphertext

'3871E275B6CD'

In [5]:
import cv2
import os

# Initialize the list to store images
a = []

# Path to the folder containing images
image_folder = 'D:\\image stego using github\\Image-Steganalysis-using-Deep-Neural-Networks\\Images\\'

# List all files in the image folder
image_files = os.listdir(image_folder)

# Filter out files that are not .jpg images
image_files = [file for file in image_files if file.lower().endswith('.jpg')]

# Sort the files to ensure sequential reading
image_files.sort()

# Fetch only the first 18 .jpg images
image_files = image_files[:18]

# Loop through each image file with an index
for i, image_file in enumerate(image_files, start=1):
    # Construct the full path to the image
    image_path = os.path.join(image_folder, image_file)
    
    # Read the image
    img = cv2.imread(image_path)
    
    # Assuming Steganography and ciphertext are defined earlier in your code
    # and the Steganography class has methods encode_data() and resize()
    filename = f'D:\\image stego using github\\Image-Steganalysis-using-Deep-Neural-Networks\\EncryptedImages\\Encrypted{i}.png'
    EncryptedImg = Steganography(img, ciphertext, filename)
    EncryptedImage = EncryptedImg.encode_data()
    EncryptedImage.resize(3, 512, 512)
    img.resize(3, 512, 512)
    a.append(EncryptedImage)
    a.append(img)


Encoded the data successfully and the image is successfully saved as  D:\image stego using github\Image-Steganalysis-using-Deep-Neural-Networks\EncryptedImages\Encrypted1.png
Encoded the data successfully and the image is successfully saved as  D:\image stego using github\Image-Steganalysis-using-Deep-Neural-Networks\EncryptedImages\Encrypted2.png
Encoded the data successfully and the image is successfully saved as  D:\image stego using github\Image-Steganalysis-using-Deep-Neural-Networks\EncryptedImages\Encrypted3.png
Encoded the data successfully and the image is successfully saved as  D:\image stego using github\Image-Steganalysis-using-Deep-Neural-Networks\EncryptedImages\Encrypted4.png
Encoded the data successfully and the image is successfully saved as  D:\image stego using github\Image-Steganalysis-using-Deep-Neural-Networks\EncryptedImages\Encrypted5.png
Encoded the data successfully and the image is successfully saved as  D:\image stego using github\Image-Steganalysis-using-De

In [6]:
a

[array([[[ 58, 114,  97, ...,  68,  53,  84],
         [ 85,  57,  94, ..., 192, 162, 246],
         [193, 160, 248, ...,  21,  36,  38],
         ...,
         [211, 190, 213, ..., 108, 112, 123],
         [ 20,  34,  30, ..., 124,  83, 133],
         [133,  92, 146, ..., 210, 205, 237]],
 
        [[213, 193, 214, ..., 102, 109, 118],
         [ 22,  36,  30, ..., 134,  93, 145],
         [145,  92, 149, ..., 213, 206, 236],
         ...,
         [ 41,  24,  39, ..., 132, 127, 108],
         [130, 125, 112, ...,  35,  40,  43],
         [ 51, 101,  83, ...,  34,  21,  35]],
 
        [[ 29,  21,  35, ..., 114, 112, 106],
         [127, 124, 109, ...,  38,  42,  43],
         [ 67, 109,  98, ...,  40,  30,  44],
         ...,
         [142, 149, 136, ...,  24,  16,  24],
         [ 23,  17,  25, ...,  40,  35,  29],
         [ 36,  31,  29, ...,  36,  33,  29]]], dtype=uint8),
 array([[[ 58, 114,  97, ...,  68,  53,  84],
         [ 85,  57,  94, ..., 192, 162, 246],
         [193, 1

In [7]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(2, 2), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=230400, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=1000, bias=True)
  (fc3): Linear(in_features=1000, out_features=2, bias=True)
  (drop1): Dropout(p=0.1, inplace=False)
  (drop2): Dropout(p=0.2, inplace=False)
  (drop3): Dropout(p=0.3, inplace=False)
  (drop4): Dropout(p=0.4, inplace=False)
  (drop5): Dropout(p=0.5, inplace=False)
  (drop6): Dropout(p=0.6, inplace=False)
)


In [8]:
a = np.array(a)
a = torch.from_numpy(a)
a = a.type(torch.FloatTensor)

In [9]:
import pandas as pd
import numpy as np
#from google.colab import files

df=pd.read_csv('Book.csv')

key_pts = df.iloc[0:36, 0:2].values

In [10]:
df

,Unnamed: 0,Unnamed: 1
0,0,1
1,1,0
2,0,1
3,1,0
4,0,1
5,1,0
6,0,1
7,1,0
8,0,1
9,1,0


In [11]:
key_pts = torch.from_numpy(key_pts)
key_pts = key_pts.type(torch.FloatTensor)

In [12]:
import torch.optim as optim
criterion = nn.MSELoss()
optimizer = optim.Adam(params = net.parameters(), lr = 0.001)

In [13]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

## **Training**

In [14]:
def train_net(n_epochs, a, key_pts):
    net.train()
    for epoch in range(n_epochs):
        running_loss = 0.0
        for i in range(len(a)):
          output_pts = net(a[i].reshape(1, 3, 512, 512))
          loss = criterion(output_pts, key_pts[i])
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          running_loss += loss.item()
          print('Epoch: {}, Batch: {}, Avg. Loss: {}'.format(epoch + 1, i+1, running_loss/10))
          running_loss = 0.0
    print('Finished Training')

In [15]:
train_net(1, a, key_pts)

Epoch: 1, Batch: 1, Avg. Loss: 0.263681697845459
Epoch: 1, Batch: 2, Avg. Loss: 3142.8015625
Epoch: 1, Batch: 3, Avg. Loss: 1235.12080078125
Epoch: 1, Batch: 4, Avg. Loss: 153.66148681640624
Epoch: 1, Batch: 5, Avg. Loss: 380.7444091796875
Epoch: 1, Batch: 6, Avg. Loss: 27.27720947265625
Epoch: 1, Batch: 7, Avg. Loss: 135.61739501953124
Epoch: 1, Batch: 8, Avg. Loss: 21.165585327148438
Epoch: 1, Batch: 9, Avg. Loss: 2.7380611419677736
Epoch: 1, Batch: 10, Avg. Loss: 1.9651126861572266
Epoch: 1, Batch: 11, Avg. Loss: 0.5043318748474122
Epoch: 1, Batch: 12, Avg. Loss: 1.4743312835693358
Epoch: 1, Batch: 13, Avg. Loss: 0.01306614577770233
Epoch: 1, Batch: 14, Avg. Loss: 0.49361133575439453
Epoch: 1, Batch: 15, Avg. Loss: 1.2061810493469238
Epoch: 1, Batch: 16, Avg. Loss: 0.6660366058349609
Epoch: 1, Batch: 17, Avg. Loss: 0.4502279281616211
Epoch: 1, Batch: 18, Avg. Loss: 1.652522087097168
Epoch: 1, Batch: 19, Avg. Loss: 0.12537460327148436
Epoch: 1, Batch: 20, Avg. Loss: 0.366690111160278

# Testing

In [16]:
image_path = 'D:\\image stego using github\\Image-Steganalysis-using-Deep-Neural-Networks\\Images\\00017.jpg'

ab = cv2.imread(image_path)
ab.resize(3, 512, 512)
ab = torch.from_numpy(ab)
ab = ab.type(torch.FloatTensor)

In [17]:
net(ab.reshape(1, 3, 512, 512))

tensor([0.2572, 0.6906], grad_fn=<ViewBackward0>)